# 📈 Metrics Analysis Notebook

This notebook analyzes **CPU** and **Memory** resource usage during performance experiments using Fortio and a service mesh (Istio or Linkerd). It reads resource usage data exported in CSV format from the Kubernetes metrics API and generates visualizations per container.

## 📁 Expected Input

- CSV files under `../results/<experiment>/metrics_<mesh>_<qps>_<payload>_<replicas>_<timestamp>.csv`
- Structure:
    - `timestamp, namespace, pod, container, cpu(n), memory(Ki)`
    - Plus derived fields: `cpu` (as float), `memory` (as float)

## 📉 Output

- CPU and Memory bar charts per container for each experiment setup.
- Output PNGs are saved in:
    - `../diagrams/<experiment>/cpu_*.png`
    - `../diagrams/<experiment>/memory_*.png`

> 📌 **Note**: CPU values are shown in nanocores, and memory in Ki.

In [25]:
# Import required libraries
import os
import glob
import pandas as pd
import numpy as np
from matplotlib import colormaps
import matplotlib.pyplot as plt
plt.style.use('default')

In [26]:
def convert_cpu(cpu_str):
    """
    Convert CPU usage strings to millicores (m).

    Conversions:
      - '123n' → nanocores → 0.000123 m
      - '123u' → microcores → 0.123 m
      - '123m' → millicores → 123.0 m
      - '123'  → cores     → 123000.0 m
    """
    try:
        s = cpu_str.lower().strip()
        if s.endswith('n'):
            value = float(s[:-1])
            return value / 1e6
        elif s.endswith('u'):
            value = float(s[:-1])
            return value / 1e3
        elif s.endswith('m'):
            return float(s[:-1])
        else:
            value = float(s)
            return value * 1000.0
    except Exception as e:
        print(f"Error converting CPU value '{cpu_str}': {e}")
        return None

def convert_memory(mem_str):
    """
    Convert memory usage from a string (which might be in Ki, Mi, or Gi) to a float representing megabytes (MB).
    
    Conversions:
    - If the value ends with "Ki": MB = (numeric value) / 1024.
    - If the value ends with "Mi": MB = numeric value.
    - If the value ends with "Gi": MB = (numeric value) * 1024.
    - If no known suffix, attempt direct conversion.
    """
    try:
        if mem_str.endswith("Ki"):
            value = float(mem_str[:-2])
            mb = value / 1024.0
            return mb
        elif mem_str.endswith("Mi"):
            value = float(mem_str[:-2])
            # Assuming 1 MiB is reported as 1 MB
            return value
        elif mem_str.endswith("Gi"):
            value = float(mem_str[:-2])
            return value * 1024.0
        else:
            # If no unit present, try converting directly
            return float(mem_str)
    except Exception as e:
        print(f"Error converting memory value '{mem_str}': {e}")
        return None

def get_bar_colors(groups):
    """
    Return a list of colors for given stacking groups.
    Specific proxy groups get fixed colors; all other groups use distinct
    entries from matplotlib's tab10 palette to avoid everything green.
    """
    proxy_map = {
        # Control Plane components
        'destination': 'tab:blue',
        'discovery': 'tab:orange',
        'identity': 'tab:green',
        'linkerd-proxy': 'tab:red',
        'policy': 'tab:purple',
        'proxy-injector': 'tab:brown',
        'sp-validator': 'tab:cyan',
        # Data-plane proxies
        'ztunnel': 'darkorange',
        'waypoint': 'orange',
        'client-proxy': 'darkblue',
        'server-proxy': 'blue'
    }
    from matplotlib import cm
    palette = colormaps['tab10']
    assigned = {}
    other_idx = 0
    colors = []
    for g in groups:
        if g in proxy_map:
            colors.append(proxy_map[g])
        else:
            if g not in assigned:
                assigned[g] = palette(other_idx % palette.N)
                other_idx += 1
            colors.append(assigned[g])
    return colors

def assign_stack_group(df):
    """
    Create a 'stack_group' column:
      - For proxies, assign one of ['ztunnel', 'waypoint', 'client-proxy', 'server-proxy']
      - For other containers, use the container name.
    """
    df = df.copy()
    conditions = [
        (df['container'] == 'istio-proxy') & df['pod'].str.contains('ztunnel'),
        (df['container'] == 'istio-proxy') & df['pod'].str.contains('waypoint'),
        df['container'].isin(['istio-proxy', 'linkerd-proxy']) & df['pod'].str.contains('client'),
        df['container'].isin(['istio-proxy', 'linkerd-proxy']) & df['pod'].str.contains('server')
    ]
    choices = ['ztunnel', 'waypoint', 'client-proxy', 'server-proxy']
    df['stack_group'] = np.select(conditions, choices, default=df['container'])
    return df

In [27]:
def load_metrics_csv(file_path):
    """
    Reads a CSV file with resource metrics, parses the timestamp, and converts the raw CPU
    (nanocores remain unchanged) and memory values (in Ki).
    
    Returns:
        A pandas DataFrame with columns:
        - timestamp
        - namespace
        - pod
        - container
        - cpu(n) (original)
        - memory(Ki) (original)
        - cpu: numeric CPU in millicore
        - memory: numeric memory in MB
    """
    df = pd.read_csv(file_path)
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['cpu'] = df['cpu(n)'].apply(convert_cpu)
    df['memory'] = df['memory(Ki)'].apply(convert_memory)
    return df

def extract_mesh_qps_payload_replicas(filename):
    """
    Extract mesh, qps, and payload from a filename assumed to be formatted as:
      metrics_<mesh>_<qps>_<payload>_<replicas>_<timestamp>.csv
    Returns a tuple: (mesh (str), qps (int), payload (int), replicas (int)).
    """
    base = os.path.basename(filename)
    parts = base.replace("metrics_", "").replace(".csv", "").split("_")
    if len(parts) < 3:
        return None, None, None
    mesh, qps, payload, replicas = parts[0], parts[1], parts[2], parts[3]
    return mesh, int(qps), int(payload), int(replicas)

def shorten_label(ns, pod, container, max_len=18):
    """
    Shortens the namespace, pod, and container names to a maximum length.
    If the name exceeds max_len, it truncates the string and appends '..'.
    Returns a formatted string: "ns/pod/container".
    """
    ns_abbr = ns if len(ns) <= max_len else ns[:max_len - 2] + '..'
    pod_abbr = pod if len(pod) <= max_len else pod[:max_len - 2] + '..'
    container_abbr = container if len(container) <= max_len else container[:max_len - 2] + '..'
    return f"{ns_abbr}/{pod_abbr}/{container_abbr}"

In [28]:
# HTTP Max throughput experiment (experiment 1)
experiment_pattern = os.path.join("..", "results", "*", "01_http_max_throughput", "metrics_*.csv")
output_dir         = os.path.join("..", "diagrams", "01_http_max_throughput")
csv_files          = glob.glob(experiment_pattern)
title              = "Experiment 1 - Max Throughput {name}\n(Prot=HTTP, QPS={qps}, Payload={payload}, Metric={metric}, Replicas={replicas})"
os.makedirs(output_dir, exist_ok=True)

if not csv_files:
    print("No CSV files found in:", experiment_pattern)
else:
    # ─── LOAD & TAG EACH ROW WITH ITS MESH ───
    df_list = []
    for file in csv_files:
        df = load_metrics_csv(file)
        mesh, qps, payload, replicas = extract_mesh_qps_payload_replicas(file)
        df['mesh'] = mesh
        df['qps']  = qps
        df['payload'] = payload
        df['replicas'] = replicas
        df_list.append(df)
    df_exp = pd.concat(df_list, ignore_index=True)
    df_exp = df_exp.dropna(subset=["mesh", "cpu", "memory"])

    # # ─── GLOBAL MAX PER CONTAINER ───
    # for metric, ylabel in [("cpu", "CPU Usage (millicore)"), ("memory", "Memory Usage (MB)")]:
    #     max_vals = df_exp.groupby(["namespace", "pod", "container"])[metric].max()
    #     labels   = [shorten_label(ns, pod, ctr) for (ns, pod, ctr) in max_vals.index]
    #     plt.figure(figsize=(15, 10))
    #     plt.bar(labels, max_vals.values,color="tab:orange", edgecolor="white", linewidth=1)
    #     plt.grid(axis="y", linestyle="--", linewidth=0.5)
    #     plt.title(title.format(name="Global", qps=qps, payload=payload, metric=metric.upper(), replicas=replicas))
    #     plt.ylabel(ylabel)
    #     plt.xticks(ha="right")
    #     plt.tight_layout()
    #     plt.savefig(os.path.join(output_dir, f"{metric}_{qps}_{payload}_{replicas}.png"))
    #     plt.close()

    for plane, name in [("control-plane", "Control Plane"), ("data-plane", "Data Plane")]:
        df_plane = df_exp[df_exp["group"] == plane].copy()
        df_plane = assign_stack_group(df_plane)
        collapsed = (df_plane.groupby(["mesh", "stack_group", "pod"]).agg({"cpu": "max", "memory": "max"}).reset_index())
        agg = (collapsed.groupby(["mesh", "stack_group"]).agg({"cpu": "sum", "memory": "sum"}).reset_index())
        for metric, ylabel in [("cpu", "CPU Usage (millicore)"), ("memory", "Memory Usage (MB)")]:

            # ───────────  STACKED CONTROL-PLANE & DATA-PLANE TOTALS  ───────────
            pivot = (agg.pivot_table(index="mesh",columns="stack_group",values=metric,fill_value=0).sort_index(axis=1))
            colors = get_bar_colors(pivot.columns)
            pivot.plot(kind="bar",stacked=True,figsize=(15, 10),color=colors,edgecolor="white",linewidth=1)
            plt.grid(axis="y", linestyle="--", linewidth=0.5)
            plt.title(title.format(name=name, qps="MAX", payload=payload, metric=metric.upper(), replicas=replicas))
            plt.ylabel(ylabel)
            plt.xticks(ha="right")
            plt.legend(title="")
            plt.tight_layout()
            suffix = "control_plane" if plane == "control-plane" else "data_plane"
            plt.savefig(os.path.join(output_dir,f"{suffix}_{metric}_{qps}_{payload}_{replicas}.png"))
            plt.close()

            # ─────────── TIMELAPSE PLOTS CUMULATIVE (Elapsed Time Per Mesh) ───────────
            metric_ts = (df_plane.groupby(['timestamp', 'mesh'])[metric].sum().reset_index())
            metric_ts['elapsed_s'] = (metric_ts.groupby('mesh')['timestamp'].transform(lambda ts: (ts - ts.min()).dt.total_seconds()))
            plt.figure(figsize=(15,10))
            for mesh_name in ['istio', 'linkerd']:
                df_m = metric_ts[metric_ts['mesh'] == mesh_name]
                plt.plot(df_m['elapsed_s'], df_m[metric], marker='o', markersize=1, linewidth=1, label=f"{mesh_name.capitalize()} - {name}")
            plt.title(title.format(name=name, qps="MAX", payload=payload, metric=metric.upper(), replicas=replicas))
            plt.xlabel("Time (seconds)")
            plt.ylabel(ylabel)
            plt.legend()
            plt.grid(axis='both', linestyle='--', linewidth=0.5)
            plt.tight_layout()
            plt.savefig(os.path.join(output_dir,f"{suffix}_timeline_cumulative_{metric}_{qps}_{payload}_{replicas}.png"))
            plt.close()

            # ─────────── TIMELAPSE PLOTS (Elapsed Time Per Mesh & Component) ───────────
            metric_ts = (df_plane.groupby(['timestamp', 'mesh', 'stack_group'])[metric].sum().reset_index())
            metric_ts['elapsed_s'] = (metric_ts.groupby('mesh')['timestamp'].transform(lambda ts: (ts - ts.min()).dt.total_seconds()))
            plt.figure(figsize=(10, 6))
            for mesh_name in ['istio', 'linkerd']:
                for comp in metric_ts.loc[metric_ts['mesh'] == mesh_name, 'stack_group'].unique():
                    df_line = metric_ts[(metric_ts['mesh'] == mesh_name) & (metric_ts['stack_group'] == comp)]
                    plt.plot(df_line['elapsed_s'], df_line[metric], marker='o', markersize=1, linewidth=1, label=f"{mesh_name.capitalize()} - {comp}")
            plt.title(title.format(name=name, qps="MAX", payload=payload, metric=metric.upper(), replicas=replicas))
            plt.xlabel("Time (seconds)")
            plt.ylabel(ylabel)
            plt.legend(ncol=2, frameon=False)
            plt.grid(axis='both', linestyle='--', linewidth=0.5)
            plt.tight_layout()
            plt.savefig(os.path.join(output_dir,f"{suffix}_timeline_{metric}_{qps}_{payload}_{replicas}.png"))
            plt.close()


In [29]:
# gRPC Max throughput experiment (experiment 2)
experiment_pattern = os.path.join("..", "results", "*", "02_grpc_max_throughput", "metrics_*.csv")
output_dir         = os.path.join("..", "diagrams", "02_grpc_max_throughput")
csv_files          = glob.glob(experiment_pattern)
title              = "Experiment 2 - Max Throughput {name}\n(Prot=gRPC, QPS={qps}, Payload={payload}, Metric={metric}, Replicas={replicas})"
os.makedirs(output_dir, exist_ok=True)

if not csv_files:
    print("No CSV files found in:", experiment_pattern)
else:
    # ─────────── LOAD & TAG MESH ───────────
    df_list = []
    for file in csv_files:
        df = load_metrics_csv(file)
        mesh, qps, payload, replicas = extract_mesh_qps_payload_replicas(file)
        df['mesh'] = mesh
        df['qps']  = qps
        df['payload'] = payload
        df['replicas'] = replicas
        df_list.append(df)
    df_exp = pd.concat(df_list, ignore_index=True)
    df_exp = df_exp.dropna(subset=["mesh", "cpu", "memory"])

    # # ─── GLOBAL MAX PER CONTAINER ───
    # for metric, ylabel in [("cpu", "CPU Usage (millicore)"), ("memory", "Memory Usage (MB)")]:
    #     max_vals = df_exp.groupby(["namespace","pod","container"])[metric].max()
    #     labels   = [shorten_label(ns, pod, ctr)for (ns, pod, ctr) in max_vals.index]
    #     plt.figure(figsize=(15,10))
    #     plt.bar(labels, max_vals.values,color="tab:orange", edgecolor="white", linewidth=1)
    #     plt.grid(axis="y", linestyle="--", linewidth=0.5)
    #     plt.title(f"Experiment 2 - Max Throughput\n"f"(Prot=gRPC, QPS={qps}, Metric={metric.upper()})")
    #     plt.ylabel(ylabel)
    #     plt.xticks(ha="right")
    #     plt.tight_layout()
    #     plt.savefig(os.path.join(output_dir, f"{metric}_{qps}_{payload}_{replicas}.png"))
    #     plt.close()

    for plane, name in [("control-plane","Control Plane"), ("data-plane","Data Plane")]:
        df_plane = df_exp[df_exp["group"] == plane].copy()
        df_plane = assign_stack_group(df_plane)
        collapsed = (df_plane.groupby(["mesh","stack_group","pod"]).agg({"cpu":"max","memory":"max"}).reset_index())
        agg = (collapsed.groupby(["mesh","stack_group"]).agg({"cpu":"sum","memory":"sum"}).reset_index())
        for metric, ylabel in [("cpu","CPU Usage (millicore)"), ("memory","Memory Usage (MB)")]:

            # ───────────  STACKED CONTROL-PLANE & DATA-PLANE TOTALS  ───────────
            pivot = (agg.pivot_table(index="mesh",columns="stack_group",values=metric,fill_value=0).sort_index(axis=1))
            colors = get_bar_colors(pivot.columns)
            pivot.plot(kind="bar",stacked=True,figsize=(15, 10),color=colors,edgecolor="white",linewidth=1)
            plt.grid(axis="y", linestyle="--", linewidth=0.5)
            plt.title(title.format(name=name, qps="MAX", payload=payload, metric=metric.upper(), replicas=replicas))
            plt.ylabel(ylabel)
            plt.xticks(ha="right")
            plt.legend(title="")
            plt.tight_layout()
            suffix = "control_plane" if plane == "control-plane" else "data_plane"
            plt.savefig(os.path.join(output_dir,f"{suffix}_{metric}_{qps}_{payload}_{replicas}.png"))
            plt.close()

            # ─────────── TIMELAPSE PLOTS CUMULATIVE (Elapsed Time Per Mesh) ───────────
            metric_ts = (df_plane.groupby(['timestamp', 'mesh'])[metric].sum().reset_index())
            metric_ts['elapsed_s'] = (metric_ts.groupby('mesh')['timestamp'].transform(lambda ts: (ts - ts.min()).dt.total_seconds()))
            plt.figure(figsize=(15,10))
            for mesh_name in ['istio', 'linkerd']:
                df_m = metric_ts[metric_ts['mesh'] == mesh_name]
                plt.plot(df_m['elapsed_s'], df_m[metric], marker='o', markersize=1, linewidth=1, label=f"{mesh_name.capitalize()} - {name}")
            plt.title(title.format(name=name, qps="MAX", payload=payload, metric=metric.upper(), replicas=replicas))
            plt.xlabel("Time (seconds)")
            plt.ylabel(ylabel)
            plt.legend()
            plt.grid(axis='both', linestyle='--', linewidth=0.5)
            plt.tight_layout()
            plt.savefig(os.path.join(output_dir,f"{suffix}_timeline_cumulative_{metric}_{qps}_{payload}_{replicas}.png"))
            plt.close()

            # ─────────── TIMELAPSE PLOTS (Elapsed Time Per Mesh & Component) ───────────
            metric_ts = (df_plane.groupby(['timestamp', 'mesh', 'stack_group'])[metric].sum().reset_index())
            metric_ts['elapsed_s'] = (metric_ts.groupby('mesh')['timestamp'].transform(lambda ts: (ts - ts.min()).dt.total_seconds()))
            plt.figure(figsize=(10, 6))
            for mesh_name in ['istio', 'linkerd']:
                for comp in metric_ts.loc[metric_ts['mesh'] == mesh_name, 'stack_group'].unique():
                    df_line = metric_ts[(metric_ts['mesh'] == mesh_name) & (metric_ts['stack_group'] == comp)]
                    plt.plot(df_line['elapsed_s'], df_line[metric], marker='o', markersize=1, linewidth=1, label=f"{mesh_name.capitalize()} - {comp}")
            plt.title(title.format(name=name, qps="MAX", payload=payload, metric=metric.upper(), replicas=replicas))
            plt.xlabel("Time (seconds)")
            plt.ylabel(ylabel)
            plt.legend(ncol=2, frameon=False)
            plt.grid(axis='both', linestyle='--', linewidth=0.5)
            plt.tight_layout()
            plt.savefig(os.path.join(output_dir,f"{suffix}_timeline_{metric}_{qps}_{payload}_{replicas}.png"))
            plt.close()


In [30]:
# HTTP Constant throughput experiment (experiment 3)
experiment_pattern = os.path.join("..", "results", "*", "03_http_constant_throughput", "metrics_*.csv")
output_dir         = os.path.join("..", "diagrams", "03_http_constant_throughput")
csv_files          = glob.glob(experiment_pattern)
title              = "Experiment 3 - Constant Throughput {name}\n(Prot=HTTP, QPS={qps}, Payload={payload}, Metric={metric}, Replicas={replicas})"
os.makedirs(output_dir, exist_ok=True)

if not csv_files:
    print("No CSV files found in:", experiment_pattern)
else:
    # ─────────── LOAD & TAG MESH + QPS ───────────
    df_list = []
    for file in csv_files:
        df = load_metrics_csv(file)
        mesh, qps, payload, replicas = extract_mesh_qps_payload_replicas(file)
        df['mesh'] = mesh
        df['qps']  = qps
        df['payload'] = payload
        df['replicas'] = replicas
        df_list.append(df)
    df_exp = pd.concat(df_list, ignore_index=True)
    df_exp = df_exp.dropna(subset=['mesh','cpu','memory','qps'])
    unique_qps = sorted(df_exp['qps'].unique())

    # # ─── GLOBAL MAX PER CONTAINER ───
    # for metric, ylabel in [('cpu','CPU Usage (millicore)'), ('memory','Memory Usage (MB)')]:
    #     for qps in unique_qps:
    #         df_q = df_exp[df_exp['qps'] == qps]
    #         max_vals = df_q.groupby(['mesh','namespace','pod','container'])[metric].max()
    #         labels   = [shorten_label(ns, pod, ctr) for (_, ns, pod, ctr) in max_vals.index]
    #         plt.figure(figsize=(15,10))
    #         plt.bar(labels, max_vals.values,color='tab:orange', edgecolor='white', linewidth=1)
    #         plt.grid(axis='y', linestyle='--', linewidth=0.5)
    #         plt.title(title.format(name=name, qps=qps, payload=payload, metric=metric.upper(), replicas=replicas))
    #         plt.ylabel(ylabel)
    #         plt.xticks(ha='right')
    #         plt.tight_layout()
    #         plt.savefig(os.path.join(output_dir, f"{metric}_{metric}_{qps}_{payload}_{replicas}.png"))
    #         plt.close()

    for plane, name in [('control-plane','Control Plane'),('data-plane','Data Plane')]:
        df_plane = df_exp[df_exp['group'] == plane].copy()
        df_plane = assign_stack_group(df_plane)
        collapsed = (df_plane.groupby(['mesh','qps','stack_group','pod']).agg({'cpu':'max','memory':'max'}).reset_index())
        agg = (collapsed.groupby(['mesh','qps','stack_group']).agg({'cpu':'sum','memory':'sum'}).reset_index())
        for qps in unique_qps:
            for metric, ylabel in [('cpu','CPU Usage (millicore)'),('memory','Memory Usage (MB)')]:

                # ───────────  STACKED CONTROL-PLANE & DATA-PLANE TOTALS  ───────────
                pivot = (agg[agg['qps'] == qps].pivot_table(index='mesh',columns='stack_group',values=metric,fill_value=0).sort_index(axis=1))
                colors = get_bar_colors(pivot.columns)
                pivot.plot(kind='bar',stacked=True,figsize=(15,10),color=colors,edgecolor='white',linewidth=1)
                plt.grid(axis='y', linestyle='--', linewidth=0.5)
                plt.title(title.format(name=name, qps=qps, payload=payload, metric=metric.upper(), replicas=replicas))
                plt.ylabel(ylabel)
                plt.xticks(ha='right')
                plt.legend(title='')
                plt.tight_layout()
                suffix = 'control_plane' if plane=='control-plane' else 'data_plane'
                plt.savefig(os.path.join(output_dir,f"{suffix}_{metric}_{qps}_{payload}_{replicas}.png"))
                plt.close()

                # ─────────── TIMELAPSE PLOTS CUMULATIVE (Elapsed Time Per Mesh) ───────────
                df_plane_q = df_plane[df_plane['qps'] == qps]
                metric_ts = (df_plane_q.groupby(['timestamp', 'mesh'])[metric].sum().reset_index())
                metric_ts['elapsed_s'] = (metric_ts.groupby('mesh')['timestamp'].transform(lambda ts: (ts - ts.min()).dt.total_seconds())
                )
                plt.figure(figsize=(15,10))
                for mesh_name in ['istio', 'linkerd']:
                    df_m = metric_ts[metric_ts['mesh'] == mesh_name]
                    plt.plot(df_m['elapsed_s'], df_m[metric], marker='o', markersize=1, linewidth=1, label=f"{mesh_name.capitalize()} - {name}")
                plt.title(title.format(name=name, qps=qps, payload=payload, metric=metric.upper(), replicas=replicas))
                plt.xlabel("Time (seconds)")
                plt.ylabel(ylabel)
                plt.legend()
                plt.grid(axis='both', linestyle='--', linewidth=0.5)
                plt.tight_layout()
                plt.savefig(os.path.join(output_dir,f"{suffix}_timeline_cumulative_{metric}_{qps}_{payload}_{replicas}.png"))
                plt.close()

                # ─────────── TIMELAPSE PLOTS (Elapsed Time Per Mesh & Component) ───────────
                metric_ts = (df_plane_q.groupby(['timestamp', 'mesh', 'stack_group'])[metric].sum().reset_index())
                metric_ts['elapsed_s'] = (metric_ts.groupby('mesh')['timestamp'].transform(lambda ts: (ts - ts.min()).dt.total_seconds()))
                plt.figure(figsize=(10,6))
                for mesh_name in ['istio', 'linkerd']:
                    for comp in metric_ts.loc[metric_ts['mesh']==mesh_name, 'stack_group'].unique():
                        df_line = metric_ts[(metric_ts['mesh'] == mesh_name) & (metric_ts['stack_group'] == comp)]
                        plt.plot(df_line['elapsed_s'], df_line[metric], marker='o', markersize=1, linewidth=1, label=f"{mesh_name.capitalize()} - {comp}")
                plt.title(title.format(name=name, qps=qps, payload=payload, metric=metric.upper(), replicas=replicas))
                plt.xlabel("Time (seconds)")
                plt.ylabel(ylabel)
                plt.legend(ncol=2, frameon=False)
                plt.grid(axis='both', linestyle='--', linewidth=0.5)
                plt.tight_layout()
                plt.savefig(os.path.join(output_dir,f"{suffix}_timeline_{metric}_{qps}_{payload}_{replicas}.png"))
                plt.close()

In [31]:
# gRPC Constant throughput experiment (experiment 4)
experiment_pattern = os.path.join("..", "results", "*", "04_grpc_constant_throughput", "metrics_*.csv")
output_dir         = os.path.join("..", "diagrams", "04_grpc_constant_throughput")
csv_files          = glob.glob(experiment_pattern)
title              = "Experiment 4 - Constant Throughput {name}\n(Prot=gRPC, QPS={qps}, Payload={payload}, Metric={metric}, Replicas={replicas})"
os.makedirs(output_dir, exist_ok=True)

if not csv_files:
    print("No CSV files found in:", experiment_pattern)
else:
    # ─────────── LOAD & TAG MESH + QPS ───────────
    df_list = []
    for file in csv_files:
        df = load_metrics_csv(file)
        mesh, qps, payload, replicas = extract_mesh_qps_payload_replicas(file)
        df['mesh'] = mesh
        df['qps']  = qps
        df['payload'] = payload
        df['replicas'] = replicas
        df_list.append(df)
    df_exp = pd.concat(df_list, ignore_index=True)
    df_exp = df_exp.dropna(subset=['mesh','cpu','memory','qps'])
    unique_qps = sorted(df_exp['qps'].unique())

    # # ─── GLOBAL MAX PER CONTAINER ───
    # for metric, ylabel in [('cpu','CPU Usage (millicore)'), ('memory','Memory Usage (MB)')]:
    #     for qps in unique_qps:
    #         df_q = df_exp[df_exp['qps'] == qps]
    #         max_vals = df_q.groupby(['mesh','namespace','pod','container'])[metric].max()
    #         labels   = [shorten_label(ns, pod, ctr) for (_, ns, pod, ctr) in max_vals.index]
    #         plt.figure(figsize=(15,10))
    #         plt.bar(labels, max_vals.values,color='tab:orange', edgecolor='white', linewidth=1)
    #         plt.grid(axis='y', linestyle='--', linewidth=0.5)
    #         plt.title(title.format(name=name, qps=qps, payload=payload, metric=metric.upper(), replicas=replicas))
    #         plt.ylabel(ylabel)
    #         plt.xticks(ha='right')
    #         plt.tight_layout()
    #         plt.savefig(os.path.join(output_dir, f"{metric}_{qps}_{payload}_{replicas}.png"))
    #         plt.close()

    for plane, name in [('control-plane','Control Plane'),('data-plane','Data Plane')]:
        df_plane = df_exp[df_exp['group'] == plane].copy()
        df_plane = assign_stack_group(df_plane)
        collapsed = (df_plane.groupby(['mesh','qps','stack_group','pod']).agg({'cpu':'max','memory':'max'}).reset_index())
        agg = (collapsed.groupby(['mesh','qps','stack_group']).agg({'cpu':'sum','memory':'sum'}).reset_index())
        for qps in unique_qps:
            for metric, ylabel in [('cpu','CPU Usage (millicore)'),('memory','Memory Usage (MB)')]:

                # ───────────  STACKED CONTROL-PLANE & DATA-PLANE TOTALS  ───────────
                pivot = (agg[agg['qps'] == qps].pivot_table(index='mesh',columns='stack_group',values=metric,fill_value=0).sort_index(axis=1))
                colors = get_bar_colors(pivot.columns)
                pivot.plot(kind='bar', stacked=True, figsize=(15,10),color=colors, edgecolor='white', linewidth=1)
                plt.grid(axis='y', linestyle='--', linewidth=0.5)
                plt.title(title.format(name=name, qps=qps, payload=payload, metric=metric.upper(), replicas=replicas))
                plt.ylabel(ylabel)
                plt.xticks(ha='right')
                plt.legend(title='')
                plt.tight_layout()
                suffix = 'control_plane' if plane=='control-plane' else 'data_plane'
                plt.savefig(os.path.join(output_dir,f"{suffix}_{metric}_{qps}_{payload}_{replicas}.png"))
                plt.close()

                # ─────────── TIMELAPSE PLOTS CUMULATIVE (Elapsed Time Per Mesh) ───────────
                df_plane_q = df_plane[df_plane['qps'] == qps]
                metric_ts = (df_plane_q.groupby(['timestamp', 'mesh'])[metric].sum().reset_index())
                metric_ts['elapsed_s'] = (metric_ts.groupby('mesh')['timestamp'].transform(lambda ts: (ts - ts.min()).dt.total_seconds()))
                plt.figure(figsize=(15,10))
                for mesh_name in ['istio', 'linkerd']:
                    df_m = metric_ts[metric_ts['mesh'] == mesh_name]
                    plt.plot(df_m['elapsed_s'], df_m[metric], marker='o', markersize=1, linewidth=1, label=f"{mesh_name.capitalize()} - {name}")
                plt.title(title.format(name=name, qps=qps, payload=payload, metric=metric.upper(), replicas=replicas))
                plt.xlabel("Time (seconds)")
                plt.ylabel(ylabel)
                plt.legend()
                plt.grid(axis='both', linestyle='--', linewidth=0.5)
                plt.tight_layout()
                plt.savefig(os.path.join(output_dir,f"{suffix}_timeline_cumulative_{metric}_{qps}_{payload}_{replicas}.png"))
                plt.close()

                # ─────────── TIMELAPSE PLOTS (Elapsed Time Per Mesh & Component) ───────────
                metric_ts = (df_plane_q.groupby(['timestamp', 'mesh', 'stack_group'])[metric].sum().reset_index())
                metric_ts['elapsed_s'] = (metric_ts.groupby('mesh')['timestamp'].transform(lambda ts: (ts - ts.min()).dt.total_seconds()))
                plt.figure(figsize=(10,6))
                for mesh_name in ['istio', 'linkerd']:
                    for comp in metric_ts.loc[metric_ts['mesh']==mesh_name, 'stack_group'].unique():
                        df_line = metric_ts[(metric_ts['mesh'] == mesh_name) & (metric_ts['stack_group'] == comp)]
                        plt.plot(df_line['elapsed_s'], df_line[metric], marker='o', markersize=1, linewidth=1, label=f"{mesh_name.capitalize()} - {comp}")
                plt.title(title.format(name=name, qps=qps, payload=payload, metric=metric.upper(), replicas=replicas))
                plt.xlabel("Time (seconds)")
                plt.ylabel(ylabel)
                plt.legend(ncol=2, frameon=False)
                plt.grid(axis='both', linestyle='--', linewidth=0.5)
                plt.tight_layout()
                plt.savefig(os.path.join(output_dir,f"{suffix}_timeline_{metric}_{qps}_{payload}_{replicas}.png"))
                plt.close()

In [32]:
# HTTP Constant throughput with Payload experiment (experiment 5)
experiment_pattern = os.path.join("..", "results", "*", "05_http_payload", "metrics_*.csv")
output_dir         = os.path.join("..", "diagrams", "05_http_payload")
csv_files          = glob.glob(experiment_pattern)
title              = "Experiment 5 - Constant Throughput with Payload {name}\n(Prot=HTTP, QPS={qps}, Payload={payload}, Metric={metric}, Replicas={replicas})"
os.makedirs(output_dir, exist_ok=True)

if not csv_files:
    print("No CSV files found in:", experiment_pattern)
else:
    # ─────────── LOAD & TAG MESH + QPS ───────────
    df_list = []
    for file in csv_files:
        df = load_metrics_csv(file)
        mesh, qps, payload, replicas = extract_mesh_qps_payload_replicas(file)
        df['mesh']    = mesh
        df['qps']     = qps
        df['payload'] = payload
        df['replicas'] = replicas
        df_list.append(df)
    df_exp = pd.concat(df_list, ignore_index=True)
    df_exp = df_exp.dropna(subset=['mesh','cpu','memory','qps','payload'])

    unique_qps     = sorted(df_exp['qps'].unique())
    unique_payload = sorted(df_exp['payload'].unique())
    qps_val = unique_qps[0] if unique_qps else None

    for payload in unique_payload:
        df_p = df_exp[df_exp['payload'] == payload]

        # # ─── GLOBAL MAX PER CONTAINER ───
        # for metric, ylabel in [('cpu','CPU Usage (millicore)'), ('memory','Memory Usage (MB)')]:
        #     max_vals = df_p.groupby(['namespace','pod','container'])[metric].max()
        #     labels   = [shorten_label(ns,pod,ctr) for ns,pod,ctr in max_vals.index]
        #     plt.figure(figsize=(15,10))
        #     plt.bar(labels, max_vals.values,color='tab:orange', edgecolor='white', linewidth=1)
        #     plt.grid(axis='y', linestyle='--', linewidth=0.5)
        #     plt.title(f"Experiment 5 - Constant Throughput\nProt=HTTP, QPS={qps_val}, Payload={payload}, Metric={metric.upper()}")
        #     plt.ylabel(ylabel)
        #     plt.xticks(ha='right')
        #     plt.tight_layout()
        #     plt.savefig(os.path.join(output_dir, f"{metric}_{qps}_{payload}_{replicas}.png"))
        #     plt.close()

        for plane, name in [('control-plane','Control Plane'), ('data-plane','Data Plane')]:
            df_plane = df_p[df_p['group'] == plane].copy()
            df_plane = assign_stack_group(df_plane)
            collapsed = (df_plane.groupby(['mesh','qps','payload','stack_group','pod']).agg({'cpu':'max','memory':'max'}).reset_index())
            agg = (collapsed.groupby(['mesh','qps','payload','stack_group']).agg({'cpu':'sum','memory':'sum'}).reset_index())
            for metric, ylabel in [('cpu','CPU Usage (millicore)'), ('memory','Memory Usage (MB)')]:

                # ───────────  STACKED CONTROL-PLANE & DATA-PLANE TOTALS  ───────────
                pivot = (agg[agg['payload']==payload].pivot_table(index='mesh',columns='stack_group',values=metric,fill_value=0).sort_index(axis=1))
                colors = get_bar_colors(pivot.columns)
                pivot.plot(kind='bar', stacked=True, figsize=(15,10),color=colors, edgecolor='white', linewidth=1)
                plt.grid(axis='y', linestyle='--', linewidth=0.5)
                plt.title(title.format(name=name, qps=qps_val, payload=payload, metric=metric.upper(), replicas=replicas))
                plt.ylabel(ylabel)
                plt.xticks(ha='right')
                plt.legend(title='')
                plt.tight_layout()
                suffix = 'control_plane' if plane=='control-plane' else 'data_plane'
                plt.savefig(os.path.join(output_dir,f"{suffix}_{metric}_{qps}_{payload}_{replicas}.png"))
                plt.close()
        
                # ─────────── TIMELAPSE PLOTS CUMULATIVE (Elapsed Time Per Mesh) ───────────
                metric_ts = (df_plane.groupby(['timestamp', 'mesh'])[metric].sum().reset_index())
                metric_ts['elapsed_s'] = (metric_ts.groupby('mesh')['timestamp'].transform(lambda ts: (ts - ts.min()).dt.total_seconds()))
                plt.figure(figsize=(15,10))
                for mesh_name in ['istio', 'linkerd']:
                    df_m = metric_ts[metric_ts['mesh'] == mesh_name]
                    plt.plot(df_m['elapsed_s'],df_m[metric], marker='o', markersize=1, linewidth=1, label=f"{mesh_name.capitalize()} - {name}")
                plt.title(title.format(name=name, qps=qps_val, payload=payload, metric=metric.upper(), replicas=replicas))
                plt.xlabel("Time (seconds)")
                plt.ylabel(ylabel)
                plt.legend()
                plt.grid(axis='both', linestyle='--', linewidth=0.5)
                plt.tight_layout()
                plt.savefig(os.path.join(output_dir,f"{suffix}_timeline_cumulative_{metric}_{qps}_{payload}_{replicas}.png"))
                plt.close()

                # ─────────── TIMELAPSE PLOTS (Elapsed Time Per Mesh & Component) ───────────
                metric_ts = (df_plane.groupby(['timestamp','mesh','stack_group'])[metric].sum().reset_index())
                metric_ts['elapsed_s'] = (metric_ts.groupby('mesh')['timestamp'].transform(lambda ts: (ts - ts.min()).dt.total_seconds()))
                plt.figure(figsize=(10, 6))
                for mesh_name in ['istio', 'linkerd']:
                    for comp in metric_ts.loc[metric_ts['mesh'] == mesh_name, 'stack_group'].unique():
                        df_line = metric_ts[(metric_ts['mesh'] == mesh_name) & (metric_ts['stack_group'] == comp)]
                        plt.plot(df_line['elapsed_s'],df_line[metric], marker='o', markersize=1, linewidth=1, label=f"{mesh_name.capitalize()} - {comp}")
                plt.title(title.format(name=name, qps=qps_val, payload=payload, metric=metric.upper(), replicas=replicas))
                plt.xlabel("Time (seconds)")
                plt.ylabel(ylabel)
                plt.legend(ncol=2, frameon=False)
                plt.grid(axis='both', linestyle='--', linewidth=0.5)
                plt.tight_layout()
                plt.savefig(os.path.join(output_dir,f"{suffix}_timeline_{metric}_{qps}_{payload}_{replicas}.png"))
                plt.close()

In [ ]:
# HTTP Constant throughput with HTTPRoute header-based routing experiment (experiment 6)
experiment_pattern = os.path.join("..", "results", "*", "06_http_constant_throughput_header", "metrics_*.csv")
output_dir         = os.path.join("..", "diagrams", "06_http_constant_throughput_header")
csv_files          = glob.glob(experiment_pattern)
title              = "Experiment 6 - Constant Throughput with HTTPRoute header-based routing {name}\n(Prot=HTTP, QPS={qps}, Payload={payload}, Metric={metric}, Replicas={replicas})"

os.makedirs(output_dir, exist_ok=True)

if not csv_files:
    print("No CSV files found in:", experiment_pattern)
else:
    # ─────────── LOAD & TAG MESH + QPS ───────────
    df_list = []
    for file in csv_files:
        df = load_metrics_csv(file)
        mesh, qps, payload, replicas = extract_mesh_qps_payload_replicas(file)
        df['mesh']    = mesh
        df['qps']     = qps
        df['payload'] = payload
        df['replicas'] = replicas
        df_list.append(df)
    df_exp     = pd.concat(df_list, ignore_index=True)
    df_exp     = df_exp.dropna(subset=["mesh", "cpu", "memory", "qps"])
    unique_qps = sorted(df_exp["qps"].unique())

    for qps in unique_qps:
        df_qps    = df_exp[df_exp["qps"] == qps]

        # # ─── GLOBAL MAX PER CONTAINER ───
        # for metric, ylabel in [("cpu", "CPU Usage (millicore)"), ("memory", "Memory Usage (MB)")]:
        #     max_vals = df_qps.groupby(["mesh", "namespace", "pod", "container"])[metric].max()
        #     labels   = [shorten_label(ns, pod, ctr) for _, ns, pod, ctr in max_vals.index]
        #     plt.figure(figsize=(15,10))
        #     plt.bar(labels, max_vals.values, color="tab:orange")
        #     plt.title(title.format(name=name, qps=qps_val, payload=payload, metric=metric.upper(), replicas=replicas))
        #     plt.ylabel(ylabel)
        #     plt.xticks(ha="right")
        #     plt.tight_layout()
        #     plt.savefig(os.path.join(output_dir, f"{metric}_{qps}_{payload}_{replicas}.png"))
        #     plt.close()
        
        for plane,name in [("control-plane","Control Plane"), ("data-plane","Data Plane")]:
            df_plane = df_qps[df_qps["group"] == plane].copy()
            df_plane = assign_stack_group(df_plane)
            collapsed = (df_plane.groupby(["mesh","qps","stack_group","pod"]).agg({"cpu":"max","memory":"max"}).reset_index())
            agg = (collapsed.groupby(["mesh","qps","stack_group"]).agg({"cpu":"sum","memory":"sum"}).reset_index())
            for metric, ylabel in [("cpu","CPU Usage (millicore)"), ("memory","Memory Usage (MB)")]:

                # ───────────  STACKED CONTROL-PLANE & DATA-PLANE TOTALS  ───────────
                pivot = (agg[agg["qps"] == qps].pivot_table(index="mesh",columns="stack_group",values=metric,fill_value=0).sort_index(axis=1))
                colors = get_bar_colors(pivot.columns)
                pivot.plot(kind="bar",stacked=True,figsize=(15,10),color=colors,edgecolor="white",linewidth=1)
                plt.grid(axis="y", linestyle="--", linewidth=0.5)
                plt.title(title.format(name=name, qps=qps_val, payload=payload, metric=metric.upper(), replicas=replicas))
                plt.ylabel(ylabel)
                plt.xticks(ha="right")
                plt.legend(title="")
                plt.tight_layout()
                suffix = "control_plane" if plane=="control-plane" else "data_plane"
                plt.savefig(os.path.join(output_dir,f"{suffix}_{metric}_{qps}_{payload}_{replicas}.png"))
                plt.close()
        
                # ─────────── TIMELAPSE PLOTS COMULATIVE (Elapsed Time Per Mesh) ───────────
                metric_ts = (df_plane.groupby(["timestamp","mesh"])[metric].sum().reset_index())
                metric_ts["elapsed_s"] = (metric_ts.groupby("mesh")["timestamp"].transform(lambda ts: (ts - ts.min()).dt.total_seconds()))
                plt.figure(figsize=(15,10))
                for mesh_name in ["istio", "linkerd"]:
                    df_m = metric_ts[metric_ts["mesh"] == mesh_name]
                    plt.plot(df_m["elapsed_s"],df_m[metric], marker="o", markersize=1, linewidth=1, label=f"{mesh_name.capitalize()} - {name}")
                plt.title(title.format(name=name, qps=qps_val, payload=payload, metric=metric.upper(), replicas=replicas))
                plt.xlabel("Time (seconds)")
                plt.ylabel(ylabel)
                plt.legend()
                plt.grid(axis="both", linestyle="--", linewidth=0.5)
                plt.tight_layout()
                plt.savefig(os.path.join(output_dir,f"{suffix}_timeline_comulative_{metric}_{qps}_{payload}_{replicas}.png"))
                plt.close()

                # ─────────── TIMELAPSE PLOTS (Elapsed Time Per Mesh) ───────────
                metric_ts = (df_plane.groupby(['timestamp', 'mesh', 'stack_group'])[metric].sum().reset_index())
                metric_ts['elapsed_s'] = metric_ts.groupby('mesh')['timestamp'].transform(lambda ts: (ts - ts.min()).dt.total_seconds())
                plt.figure(figsize=(10, 6))
                for mesh_name in ['istio', 'linkerd']:
                    comps = metric_ts.loc[metric_ts['mesh'] == mesh_name, 'stack_group'].unique()
                    for comp in comps:
                        df_line = metric_ts[(metric_ts['mesh'] == mesh_name) &(metric_ts['stack_group'] == comp)]
                        plt.plot(df_line['elapsed_s'],df_line[metric], marker='o', markersize=1, linewidth=1, label=f"{mesh_name.capitalize()} - {comp}")
                plt.title(title.format(name=name, qps=qps_val, payload=payload, metric=metric.upper(), replicas=replicas))
                plt.xlabel("Time (seconds)")
                plt.ylabel(ylabel)
                plt.legend(ncol=2, frameon=False)
                plt.grid(axis='both', linestyle='--', linewidth=0.5)
                plt.tight_layout()
                plt.savefig(os.path.join(output_dir,f"{suffix}_timeline_{metric}_{qps}_{payload}_{replicas}.png"))
                plt.close()

In [ ]:
# # HTTP Constant throughput with multiple replicas experiment (experiment 7)
experiment_pattern = os.path.join("..", "results", "*", "07_resource_consumption", "metrics_*.csv")
output_dir         = os.path.join("..", "diagrams", "07_resource_consumption")
csv_files          = glob.glob(experiment_pattern)
title              = "Experiment 7 - Constant Throughput with Multiple Replicas {name}\n(Prot=HTTP, QPS={qps}, Payload={payload}, Metric={metric}, Replicas={replicas})"

os.makedirs(output_dir, exist_ok=True)

if not csv_files:
    print("No CSV files found in:", experiment_pattern)
else:
    # ─────────── LOAD & TAG ───────────
    df_list = []
    for file in csv_files:
        df = load_metrics_csv(file)
        mesh, qps, payload, replicas = extract_mesh_qps_payload_replicas(file)
        df['mesh']     = mesh
        df['qps']      = qps
        df['payload']  = payload
        df['replicas'] = replicas
        df_list.append(df)
    df_exp = pd.concat(df_list, ignore_index=True)
    df_exp = df_exp.dropna(subset=['cpu', 'memory', 'mesh', 'qps', 'payload'])
    unique_qps      = int(df_exp["qps"].unique()[0])
    unique_payload  = int(df_exp["payload"].unique()[0])
    unique_replicas = int(df_exp["replicas"].unique()[0])

    for plane, plane_name in [("control-plane", "Control Plane"), ("data-plane", "Data Plane")]:
        df_plane = df_exp[df_exp['group'] == plane].copy()
        df_plane = assign_stack_group(df_plane)
        collapsed = (df_plane.groupby(['mesh', 'stack_group', 'pod']).agg({'cpu': 'max', 'memory': 'max'}).reset_index())
        agg = (collapsed.groupby(['mesh', 'stack_group']).agg({'cpu': 'sum', 'memory': 'sum'}).reset_index())
        for metric, ylabel in [('cpu', 'CPU Usage (millicore)'), ('memory', 'Memory Usage (MB)')]:
            
            # ───────────  STACKED CONTROL-PLANE & DATA-PLANE TOTALS  ───────────
            pivot = (agg.pivot_table(index='mesh', columns='stack_group', values=metric, fill_value=0).sort_index(axis=1))
            colors = get_bar_colors(pivot.columns)
            pivot.plot(kind='bar', stacked=True, figsize=(10, 6), color=colors, edgecolor='white', linewidth=1)
            plt.grid(axis='y', linestyle='--', linewidth=0.5)
            plt.title(title.format(name=name, qps=unique_qps, payload=unique_payload, metric=metric.upper(), replicas=unique_replicas))
            plt.ylabel(ylabel)
            plt.xticks(rotation=0)
            plt.legend(title='')
            plt.tight_layout()
            suffix = "control_plane" if plane=="control-plane" else "data_plane"
            plt.savefig(os.path.join(output_dir,f"{suffix}_{metric}_{qps}_{payload}_{replicas}.png"))
            plt.close()

            # ─────────── TIMELAPSE PLOTS COMULATIVE (Elapsed Time Per Mesh) ───────────
            metric_ts = (df_plane.groupby(['timestamp', 'mesh'])[metric].sum().reset_index())
            metric_ts['elapsed_s'] = (metric_ts.groupby('mesh')['timestamp'].transform(lambda ts: (ts - ts.min()).dt.total_seconds()))
            plt.figure(figsize=(10, 6))
            for mesh_name in ['istio', 'linkerd']:
                df_m = metric_ts[metric_ts['mesh'] == mesh_name]
                plt.plot(df_m['elapsed_s'], df_m[metric], marker='o', markersize=1, linewidth=1, label=mesh_name.capitalize())
            plt.title(title.format(name=name, qps=unique_qps, payload=unique_payload, metric=metric.upper(), replicas=unique_replicas))
            plt.xlabel("Time (seconds)")
            plt.ylabel(ylabel)
            plt.legend()
            plt.grid(axis='both', linestyle='--', linewidth=0.5)
            plt.tight_layout()
            plt.savefig(os.path.join(output_dir,f"{suffix}_timeline_comulative_{metric}_{unique_qps}_{unique_payload}_{unique_replicas}.png"))
            plt.close()

            # ─────────── TIMELAPSE PLOTS (Elapsed Time Per Mesh) ───────────
            metric_ts = (df_plane.groupby(['timestamp', 'mesh', 'stack_group'])[metric].sum().reset_index())
            metric_ts['elapsed_s'] = (metric_ts.groupby('mesh')['timestamp'].transform(lambda ts: (ts - ts.min()).dt.total_seconds()))
            plt.figure(figsize=(10, 6))
            for mesh_name in ['istio', 'linkerd']:
                for comp in metric_ts.loc[metric_ts['mesh'] == mesh_name, 'stack_group'].unique():
                    df_line = metric_ts[(metric_ts['mesh'] == mesh_name) & (metric_ts['stack_group'] == comp)]
                    plt.plot(df_line['elapsed_s'], df_line[metric], marker='o', markersize=1, linewidth=1, label=f"{mesh_name.capitalize()} - {comp}")
            plt.title(title.format(name=name, qps=unique_qps, payload=unique_payload, metric=metric.upper(), replicas=unique_replicas))
            plt.xlabel("Time (seconds)")
            plt.ylabel(ylabel)
            plt.legend(ncol=2, frameon=False)
            plt.grid(axis='both', linestyle='--', linewidth=0.5)
            plt.tight_layout()
            plt.savefig(os.path.join(output_dir,f"{suffix}_timeline_{metric}_{unique_qps}_{unique_payload}_{unique_replicas}.png"))
            plt.close()